In [1]:
import pandas as pd

dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

sales = pd.read_csv('kc_house_data.csv', dtype=dtype_dict)

In [2]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']
sales['floors_square'] = sales['floors']*sales['floors']

In [3]:
from sklearn import linear_model  # using scikit-learn

all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

model_all = linear_model.Lasso(alpha=5e2, normalize=True) # set parameters
model_all.fit(sales[all_features], sales['price']) # learn weights

Lasso(alpha=500.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=True, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [4]:
print(model_all.intercept_)

-218136.21403514023


In [5]:
print(model_all.coef_)

[    0.             0.             0.           134.43931396
     0.             0.             0.             0.
     0.             0.         24750.00458561     0.
 61749.10309071     0.             0.            -0.
     0.        ]


In [6]:
coef_dict = dict(zip(all_features, model_all.coef_))


In [7]:
coef_dict

{'bedrooms': 0.0,
 'bedrooms_square': 0.0,
 'bathrooms': 0.0,
 'sqft_living': 134.43931395541443,
 'sqft_living_sqrt': 0.0,
 'sqft_lot': 0.0,
 'sqft_lot_sqrt': 0.0,
 'floors': 0.0,
 'floors_square': 0.0,
 'waterfront': 0.0,
 'view': 24750.00458560952,
 'condition': 0.0,
 'grade': 61749.10309070802,
 'sqft_above': 0.0,
 'sqft_basement': 0.0,
 'yr_built': -0.0,
 'yr_renovated': 0.0}

In [8]:
testing = pd.read_csv('wk3_kc_house_test_data.csv', dtype=dtype_dict)
training = pd.read_csv('wk3_kc_house_train_data.csv', dtype=dtype_dict)
validation = pd.read_csv('wk3_kc_house_valid_data.csv', dtype=dtype_dict)

In [9]:
testing['sqft_living_sqrt'] = testing['sqft_living'].apply(sqrt)
testing['sqft_lot_sqrt'] = testing['sqft_lot'].apply(sqrt)
testing['bedrooms_square'] = testing['bedrooms']*testing['bedrooms']
testing['floors_square'] = testing['floors']*testing['floors']

training['sqft_living_sqrt'] = training['sqft_living'].apply(sqrt)
training['sqft_lot_sqrt'] = training['sqft_lot'].apply(sqrt)
training['bedrooms_square'] = training['bedrooms']*training['bedrooms']
training['floors_square'] = training['floors']*training['floors']

validation['sqft_living_sqrt'] = validation['sqft_living'].apply(sqrt)
validation['sqft_lot_sqrt'] = validation['sqft_lot'].apply(sqrt)
validation['bedrooms_square'] = validation['bedrooms']*validation['bedrooms']
validation['floors_square'] = validation['floors']*validation['floors']

In [10]:
validation_rss={}
import numpy as np
for l1_penalty in np.logspace(1,7,num=13):
    model = linear_model.Lasso(alpha=l1_penalty, normalize=True)
    model_1=model.fit(training[all_features],training['price'])
    prediction=model_1.predict(validation[all_features])
    error=prediction-validation['price']
    RSS=sum(error**2)
    validation_rss[l1_penalty]=RSS
print(validation_rss)
print (min(validation_rss.items(), key=lambda x: x[1]))

{10.0: 398213327300134.4, 31.622776601683793: 399041900253347.8, 100.0: 429791604072558.25, 316.22776601683796: 463739831045119.7, 1000.0: 645898733633803.5, 3162.2776601683795: 1222506859427156.8, 10000.0: 1222506859427156.8, 31622.776601683792: 1222506859427156.8, 100000.0: 1222506859427156.8, 316227.7660168379: 1222506859427156.8, 1000000.0: 1222506859427156.8, 3162277.6601683795: 1222506859427156.8, 10000000.0: 1222506859427156.8}
(10.0, 398213327300134.4)


In [11]:
model = linear_model.Lasso(alpha=10, normalize=True)
model_test=model.fit(training[all_features],training['price'])
prediction=model_test.predict(testing[all_features])
error=prediction-testing['price']
RSS=sum(error**2)
print(RSS)


98467402552698.88


In [12]:
np.count_nonzero(model_test.coef_) + np.count_nonzero(model_test.intercept_)


15

In [13]:
coef_dict = dict(zip(all_features, model_test.coef_))
coef_dict


{'bedrooms': -16144.56275707671,
 'bedrooms_square': 373.2453843491054,
 'bathrooms': 50841.2433398644,
 'sqft_living': 617.8535595042771,
 'sqft_living_sqrt': -44411.3548667116,
 'sqft_lot': 0.7856230648316205,
 'sqft_lot_sqrt': -701.1947653681414,
 'floors': -0.0,
 'floors_square': 5014.200456968523,
 'waterfront': 619488.7524859126,
 'view': 38041.85565247407,
 'condition': 24998.771838159242,
 'grade': 128716.234621461,
 'sqft_above': 0.0,
 'sqft_basement': 0.0,
 'yr_built': -3293.831179945395,
 'yr_renovated': 10.057320864287924}

In [14]:
model_test.coef_

array([-1.61445628e+04,  3.73245384e+02,  5.08412433e+04,  6.17853560e+02,
       -4.44113549e+04,  7.85623065e-01, -7.01194765e+02, -0.00000000e+00,
        5.01420046e+03,  6.19488752e+05,  3.80418557e+04,  2.49987718e+04,
        1.28716235e+05,  0.00000000e+00,  0.00000000e+00, -3.29383118e+03,
        1.00573209e+01])

In [15]:
coef_dict={}
import numpy as np
for l1_penalty in np.logspace(1,4,num=20):
    model = linear_model.Lasso(alpha=l1_penalty, normalize=True)
    model_1=model.fit(training[all_features],training['price'])
    non_zero_coef=np.count_nonzero(model_1.coef_) + np.count_nonzero(model_1.intercept_)
    coef_dict[l1_penalty]=non_zero_coef
    
print(coef_dict)

{10.0: 15, 14.38449888287663: 15, 20.6913808111479: 15, 29.76351441631318: 15, 42.81332398719393: 13, 61.58482110660264: 12, 88.58667904100822: 11, 127.42749857031335: 10, 183.29807108324357: 7, 263.6650898730358: 6, 379.26901907322497: 6, 545.5594781168514: 6, 784.7599703514607: 5, 1128.8378916846884: 3, 1623.776739188721: 3, 2335.7214690901214: 2, 3359.818286283781: 1, 4832.930238571752: 1, 6951.927961775606: 1, 10000.0: 1}


In [16]:
validation_rss={}
import numpy as np
for l1_penalty in np.linspace(127.42749857031335,263.6650898730358,num=20):
    model = linear_model.Lasso(alpha=l1_penalty, normalize=True)
    model_1=model.fit(training[all_features],training['price'])
    prediction=model_1.predict(validation[all_features])
    error=prediction-validation['price']
    RSS=sum(error**2)
    non_zero_coef=np.count_nonzero(model_1.coef_) + np.count_nonzero(model_1.intercept_)
    validation_rss[l1_penalty]=RSS,non_zero_coef

print(validation_rss)


{127.42749857031335: (435374677102679.3, 10), 134.5978981125619: (437009229124473.5, 10), 141.76829765481045: (438236128386911.3, 8), 148.938697197059: (439158937799659.44, 8), 156.10909673930755: (440037365263316.94, 7), 163.2794962815561: (440777489641605.25, 7), 170.44989582380464: (441566698090138.9, 7), 177.6202953660532: (442406413188665.56, 7), 184.79069490830176: (443296716874312.9, 7), 191.96109445055032: (444239780526140.75, 7), 199.13149399279888: (445230739842614.1, 7), 206.3018935350474: (446268896864705.5, 6), 213.47229307729594: (447112919434642.25, 6), 220.6426926195445: (447998187851566.4, 6), 227.81309216179307: (448924706673256.0, 6), 234.98349170404163: (449892475899710.9, 6), 242.1538912462902: (450901498778122.44, 6), 249.32429078853872: (451952426654984.1, 6), 256.49469033078725: (453043924367600.6, 6), 263.6650898730358: (454176669662633.8, 6)}


In [17]:
validation={}
non_zero_coef==7
validation[l1_penalty]=RSS,non_zero_coef


In [18]:
validation

{263.6650898730358: (454176669662633.8, 6)}

In [19]:

model = linear_model.Lasso(alpha=156.10909673930755, normalize=True)
model_1=model.fit(training[all_features],training['price'])
non_zero_coef=np.count_nonzero(model_1.coef_) + np.count_nonzero(model_1.intercept_)
coef_dict = dict(zip(all_features, model_1.coef_))
coef_dict


{'bedrooms': -0.0,
 'bedrooms_square': -0.0,
 'bathrooms': 10610.890284398312,
 'sqft_living': 163.38025164762888,
 'sqft_living_sqrt': 0.0,
 'sqft_lot': -0.0,
 'sqft_lot_sqrt': -0.0,
 'floors': 0.0,
 'floors_square': 0.0,
 'waterfront': 506451.68711484916,
 'view': 41960.04355485288,
 'condition': 0.0,
 'grade': 116253.5536997075,
 'sqft_above': 0.0,
 'sqft_basement': 0.0,
 'yr_built': -2612.2348803574873,
 'yr_renovated': 0.0}